# <font color='blue'>Data Science Academy - Machine Learning</font>

# <font color='blue'>Capítulo 12 - Processamento de Linguagem Natural</font>

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Obs: Este é um material de bônus incluído neste curso. PyTorch é estudado em detalhes no curso <a href="https://www.datascienceacademy.com.br/course?courseid=deep-learning-frameworks">Deep Learning Frameworks</a> e aplicado em PLN no curso <a href="https://www.datascienceacademy.com.br/course?courseid=processamento-de-linguagem-natural-e-reconhecimento-de-voz">Processamento de Linguagem Natural</a>.

### Estudo de Caso - Inteligência Artificial Para Previsão de Sentenças em Embargos de Declaração

![title](imagens/embargos.png)

**A definição deste estudo de caso está no manual em pdf no Capítulo 12 do Curso de <a href="https://www.datascienceacademy.com.br/course?courseid=machine-learning-engineer">Machine Learning</a>**. 

Faça a leitura do manual antes de prosseguir com o Estudo de Caso.

In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install torch==1.5.0

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Instala o PyTorch
!pip install -q torch 

In [4]:
# Imports
import torch
import torch.nn as nn
import numpy as np

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

torch 1.5.0
numpy 1.18.2
Data Science Academy


### Preparando os Dados

O texto abaixo é um exemplo de embargo de declaração. Embora o texto represente um embargo, dados críticos foram substituídos por informações genéricas, o que não compromete o objetivo do estudo de caso.

In [6]:
# Texto de embargo de declaração
embargo = """O embargante sofreu o ajuizamento de ação de danos morais e materiais, cujo objeto é o reaver os 
valores pagos pelo sinal dado em um contrato de compra e venda de imóvel no qual não foi dado continuidade. Em 24 
de fevereiro de 2012, o Magistrado proferiu decisão de fls. 277 a 280, que condenou todas as demandadas 
solidariamente no seguinte teor: Diante de todo o exposto, com fundamento no art. 1234, I, do CPC/2015, 
julgo procedentes em parte os pedidos constantes na inicial, condenando solidariamente as demandadas, XPTO LTDA, 
BOB CAMARGO DE MORAES, a Pagarema título de indenização por danos morais, consoante fundamentação acima discorrida, 
o montante de R$ 1.500,00 (um mil e quinhentos reais), corrigidos monetariamente pelo INPC desde a data 
desta decisão, acrescidos de juros de 1% ao mês, a partir da citação; condeno ainda, à restituição do valor 
pago pelo demandante como sinal da entrada do imóvel, descontando apenas 20% (vinte por cento), referente às 
despesas, devendo incidir juros de 1% (um por cento) ao mês contados da citação e correção monetária pelo INPC a 
partir da sentença. Contudo, data venia, houve omissão e obscuridade na referida decisão, haja vista que a omissão 
se deu pela ausência dos julgamentos das preliminares (Necessidade de Perícia Técnica e a incompetência de 
Juizado Especial) proposta posteriormente em aditamento de contestação (Fls 251 a 254) para impugnar áudios 
juntados pelo embargado, autorizado a ser realizada pela Douta Magistrada em audiência de Conciliação, 
instrução e julgamento de fls 235 e 236, por ausência de intimação anterior para realizar a já tratada 
impugnação aos áudios anexados."""

In [7]:
# Limpeza do texto substituindo vírgulas e pontos por espaços e colocando as palavras em minúsculo
embargo = embargo.replace(',','').replace('.','').lower().split()

In [8]:
# Criação do corpus com o texto acima
corpus = set(embargo)

In [9]:
# Visualizamos o corpus
corpus

{'(fls',
 '(necessidade',
 '(um',
 '(vinte',
 '1%',
 '1234',
 '150000',
 '20%',
 '2012',
 '235',
 '236',
 '24',
 '251',
 '254)',
 '277',
 '280',
 'a',
 'acima',
 'acrescidos',
 'aditamento',
 'ainda',
 'ajuizamento',
 'anexados',
 'anterior',
 'ao',
 'aos',
 'apenas',
 'art',
 'as',
 'audiência',
 'ausência',
 'autorizado',
 'ação',
 'bob',
 'camargo',
 'cento)',
 'citação',
 'citação;',
 'com',
 'como',
 'compra',
 'conciliação',
 'condenando',
 'condeno',
 'condenou',
 'consoante',
 'constantes',
 'contados',
 'contestação',
 'continuidade',
 'contrato',
 'contudo',
 'correção',
 'corrigidos',
 'cpc/2015',
 'cujo',
 'da',
 'dado',
 'danos',
 'das',
 'data',
 'de',
 'decisão',
 'demandadas',
 'demandante',
 'descontando',
 'desde',
 'despesas',
 'desta',
 'deu',
 'devendo',
 'diante',
 'discorrida',
 'do',
 'dos',
 'douta',
 'e',
 'em',
 'embargado',
 'embargante',
 'entrada',
 'especial)',
 'exposto',
 'fevereiro',
 'fls',
 'foi',
 'fundamentação',
 'fundamento',
 'haja',
 'houve',
 

In [10]:
# Comprimento do corpus
corpus_length = len(corpus)

In [11]:
# Dicionários para TF-IDF
dic_palavra = {}
dic_inverso_palavra = {}

In [12]:
# Loop pelo corpus para criar os dicionários
for i, palavra in enumerate(corpus):
    dic_palavra[palavra] = i
    dic_inverso_palavra[i] = palavra

In [13]:
# Lista para receber os dados
dados = []

In [14]:
# Loop pelo texto par extrair sentenças e palavras
for i in range(2, len(embargo) - 2):
    sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]
    target = embargo[i]
    dados.append((sentence, target))

Você leu o código acima e compreendeu o que foi feito? Observe esta linha:

sentence = [embargo[i-2], embargo[i-1], embargo[i+1], embargo[i+2]]

Para uma palavra no índice i, obtemos duas palavras antes e duas palavras depois. A palavra no índice i será o nosso target e a sentença será composta das duas palavras e duas palavras depois da palavra target. 

Após treinar o modelo, seremos capazes de prever cada palavra com base nas palavras a sua volta.

Aqui um exemplo:

In [15]:
# Visualiza os dados
print(dados[3])

(['o', 'ajuizamento', 'ação', 'de'], 'de')


As quatro palavras na lista serão os dados de entrada e a palavra fora da lista ('de' nesse caso), será a variável de saída.

### Construção do Modelo CBoW

In [16]:
# Vamos definir o comprimento de cada embedding
embedding_length = 20

In [17]:
# Classe para o modelo
class CBoW(torch.nn.Module):

    # Método construtor
    def __init__(self, corpus_length, embedding_dim):
        super(CBoW, self).__init__()
        
        # Camada de entrada do modelo para criação da embedding
        self.embeddings = nn.Embedding(corpus_length, embedding_dim)

        # Camadas lineares
        self.linear1 = nn.Linear(embedding_dim, 64)
        self.linear2 = nn.Linear(64, corpus_length)
        
        # Camadas de ativação
        self.activation_function1 = nn.ReLU()
        self.activation_function2 = nn.LogSoftmax(dim = -1)

    # Passo (forward)
    def forward(self, inputs):
        
        # Aqui definimos a ordem ds camadas da rede neural
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    # Obtém a word_emdedding
    def get_word_emdedding(self, word):
        word = torch.LongTensor([dic_palavra[word]])
        return self.embeddings(word).view(1,-1)

In [18]:
# Cria o modelo CBoW
modelo = CBoW(corpus_length, embedding_length)

In [19]:
# Função de custo
loss_function = nn.NLLLoss()

In [20]:
# Otimizador do modelo (backpropagation)
optimizer = torch.optim.SGD(modelo.parameters(), lr = 0.01)

In [21]:
# Função para criar o vetor de sentenças, necessário para treinar o modelo
def make_sentence_vector(sentence, word_dict):
    idxs = [word_dict[w] for w in sentence]
    return torch.tensor(idxs, dtype = torch.long)

In [22]:
# Aqui está nosso dicionário de palavras
dic_palavra

{'20%': 0,
 'para': 1,
 'realizar': 2,
 'acrescidos': 3,
 'incompetência': 4,
 'julgamentos': 5,
 'juizado': 6,
 'perícia': 7,
 '251': 8,
 'montante': 9,
 'obscuridade': 10,
 'omissão': 11,
 'venda': 12,
 '236': 13,
 'corrigidos': 14,
 'na': 15,
 'embargado': 16,
 'pagarema': 17,
 'posteriormente': 18,
 'sinal': 19,
 'um': 20,
 'aos': 21,
 'preliminares': 22,
 '(um': 23,
 'pago': 24,
 'dos': 25,
 'houve': 26,
 '1%': 27,
 'quinhentos': 28,
 'demandadas': 29,
 'mês': 30,
 'condenou': 31,
 'discorrida': 32,
 'pela': 33,
 'como': 34,
 'valores': 35,
 'no': 36,
 '2012': 37,
 'partir': 38,
 '280': 39,
 'impugnação': 40,
 'inpc': 41,
 'correção': 42,
 'áudios': 43,
 'desde': 44,
 'restituição': 45,
 'às': 46,
 'ajuizamento': 47,
 'reais)': 48,
 'solidariamente': 49,
 'data': 50,
 'tratada': 51,
 'haja': 52,
 'apenas': 53,
 'impugnar': 54,
 'diante': 55,
 '(vinte': 56,
 'julgamento': 57,
 'compra': 58,
 'seguinte': 59,
 'cento)': 60,
 'demandante': 61,
 'descontando': 62,
 'contados': 63,
 'se

In [23]:
# O dicionário de palavras será convertido em um vetor de sentenças. Aqui um exemplo:
print(make_sentence_vector(['pela','ausência','dos','julgamentos'], dic_palavra))

tensor([ 33, 102,  25,   5])


### Treinamento do Modelo

In [24]:
# Loop por 150 passadas (epochs) de treinamento
for epoch in range(150):
    
    # Inicia o erro da época com 0
    epoch_loss = 0
    
    # Loop pelos dados de entrada (sentence) e saída (target)
    for sentence, target in dados:
        
        # Inicializa os gradientes com zero
        modelo.zero_grad()
        
        # Cria o vetor de sentença com os dados de entrada (que devem estar no dicionário de palavras)
        sentence_vector = make_sentence_vector(sentence, dic_palavra)  
        
        # Usa o vetor para fazer previsões com o modelo e retorna as probabilidades
        log_probs = modelo(sentence_vector)
        
        # Calcula o erro do modelo
        loss = loss_function(log_probs, torch.tensor([dic_palavra[target]], dtype = torch.long))
        
        # Chama o método de backpropagation para calcular o gradiente da derivada
        loss.backward()
        
        # Otimiza os pesos do modelo e segue para a próxima passada
        # É aqui que o aprendizado acontece
        optimizer.step()
        
        # Atualiza o erro da época
        epoch_loss += loss.data
        
    # Imprime epoch e erro da epoch    
    print('Epoch: ' + str(epoch) + ', Erro do Modelo: ' + str(epoch_loss.item()))

Epoch: 0, Erro do Modelo: 1349.9468994140625
Epoch: 1, Erro do Modelo: 1222.0167236328125
Epoch: 2, Erro do Modelo: 1123.184814453125
Epoch: 3, Erro do Modelo: 1026.185546875
Epoch: 4, Erro do Modelo: 923.4420166015625
Epoch: 5, Erro do Modelo: 814.168701171875
Epoch: 6, Erro do Modelo: 699.1925048828125
Epoch: 7, Erro do Modelo: 583.171142578125
Epoch: 8, Erro do Modelo: 470.5594177246094
Epoch: 9, Erro do Modelo: 366.5306091308594
Epoch: 10, Erro do Modelo: 276.91217041015625
Epoch: 11, Erro do Modelo: 204.39186096191406
Epoch: 12, Erro do Modelo: 150.15167236328125
Epoch: 13, Erro do Modelo: 110.65082550048828
Epoch: 14, Erro do Modelo: 84.32002258300781
Epoch: 15, Erro do Modelo: 65.56706237792969
Epoch: 16, Erro do Modelo: 52.40394973754883
Epoch: 17, Erro do Modelo: 43.125732421875
Epoch: 18, Erro do Modelo: 35.943904876708984
Epoch: 19, Erro do Modelo: 30.393281936645508
Epoch: 20, Erro do Modelo: 25.812850952148438
Epoch: 21, Erro do Modelo: 21.962703704833984
Epoch: 22, Erro d

Observe como o erro foi reduzido a cada passada, nitidamente o aprendizado ocorrendo. Vamos agora usar o modelo para fazer previsões.

In [25]:
# Função para obter uma previsão
def get_resultado_previsto(input, dic_inverso_palavra):
    index = np.argmax(input)
    return dic_inverso_palavra[index]

In [26]:
# Função para prever sentenças (aplicamos aos novos dados o mesmo tratamento usado nos dados de treino)
def preve_sentenca(sentence):
    
    # Dividimos a sentença com split
    sentence_split = sentence.replace('.','').lower().split()
    
    # Criamos o vetor de sentença
    sentence_vector = make_sentence_vector(sentence_split, dic_palavra)
    
    # Faz a previsão com o modelo
    prediction_array = modelo(sentence_vector).data.numpy()
    
    # Print dos resultados
    print('Palavras Anteriores: {}\n'.format(sentence_split[:2]))
    print('Palavra Prevista: {}\n'.format(get_resultado_previsto(prediction_array[0], dic_inverso_palavra)))
    print('Palavras Seguintes: {}\n'.format(sentence_split[2:]))

### Previsões com o Modelo

Dentro da frase: **"ausência de intimação anterior para realizar"**, vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra **intimação** e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [27]:
# Previsão com o modelo
preve_sentenca('ausência de anterior para')

Palavras Anteriores: ['ausência', 'de']

Palavra Prevista: intimação

Palavras Seguintes: ['anterior', 'para']



In [28]:
# Emdedding da palavra
print(modelo.get_word_emdedding('intimação'))

tensor([[-0.9246,  0.2517, -0.6917,  0.1016, -0.5898, -0.6693,  0.6703,  1.3784,
          1.2231, -0.8615, -0.5857,  0.3420, -0.7337, -1.0864,  0.9474,  0.7901,
          0.4599,  0.5518, -1.0639,  1.0161]], grad_fn=<ViewBackward>)


Perfeito! O modelo fez a previsão da sentença no Embargo de Declaração! Mais um exemplo.

Dentro da frase: **"devendo incidir juros de 1%"**, vejamos se o modelo consegue prever a palavra.

Vou omitir a palavra **juros** e essa deve ser a palavra prevista pelo modelo. Vamos passar como dados de entrada as duas palavras anteriores e as duas palavras posteriores.

In [29]:
# Previsão com o modelo
preve_sentenca('devendo incidir de 1%')

Palavras Anteriores: ['devendo', 'incidir']

Palavra Prevista: juros

Palavras Seguintes: ['de', '1%']



Perfeito! O modelo fez a previsão da sentença no Embargo de Declaração! E o CBoW não é o modelo mais avançado em PLN.

# Fim